In [ ]:
%cd /content/drive/MyDrive/new_art

/content/drive/MyDrive/new_art


In [ ]:

!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 8.0 MB/s eta 0:00:00


In [ ]:
from catboost import CatBoostRegressor
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import pandas as pd
import numpy as np

In [ ]:
# org_data = pd.read_excel('TalkFile_df.xlsx')
org_data = pd.read_csv('test_df.csv')  ##custom training data

In [ ]:
new_data = org_data.copy()

In [ ]:
ex_frame = 20

In [ ]:
new_data

,index,x,y,V,bounce,hit
0,0,764.000000,572.000000,0.000000,0,0
1,1,764.000000,572.000000,0.000000,0,0
2,2,764.000000,572.000000,0.000000,0,0
3,3,764.000000,572.000000,0.000000,0,0
4,4,764.000000,572.000000,0.000000,0,0
...,...,...,...,...,...,...
13348,13348,536.000000,200.000000,10.239241,0,0
13349,13349,537.000000,203.000000,4.252899,0,0
13350,13350,537.333333,204.333333,4.554660,0,0
13351,13351,537.666667,205.666667,4.552513,0,0


In [ ]:
for i in range(ex_frame, 0, -1):
  new_data[f'lagX_{i}'] = new_data['x'].shift(i, fill_value=0)
for i in range(ex_frame, 0, -1):
  new_data[f'lagY_{i}'] = new_data['y'].shift(i, fill_value=0)
for i in range(ex_frame, 0, -1):
  new_data[f'lagV_{i}'] = new_data['V'].shift(i, fill_value=0)

In [ ]:
new_bounce = []

for i in range(len(new_data)):
  if i < ex_frame//2:
    new_bounce.append(0)
  else:
    new_bounce.append(new_data['bounce'][i-ex_frame//2])

new_data['new_bounce'] = new_bounce

In [ ]:
# 'lagX_20', 'lagX_19', 'lagX_18', 'lagX_17', 'lagX_16',
#       'lagX_15', 'lagX_14', 'lagX_13', 'lagX_12', 'lagX_11', 'lagX_10',
      # 'lagX_9', 'lagX_8', 'lagX_7', 'lagX_6', 'lagX_5',

#  'lagY_20', 'lagY_19', 'lagY_18', 'lagY_17',
#       'lagY_16', 'lagY_15', 'lagY_14', 'lagY_13', 'lagY_12', 'lagY_11',
# 'lagY_10', 'lagY_9', 'lagY_8', 'lagY_7', 'lagY_6', 'lagY_5',

# 'lagV_20', 'lagV_19', 'lagV_18',
#       'lagV_17', 'lagV_16', 'lagV_15', 'lagV_14', 'lagV_13', 'lagV_12',
#       'lagV_11','lagV_10', 'lagV_9', 'lagV_8', 'lagV_7', 'lagV_6', 'lagV_5',

# new_data_co = new_data[[ 'lagX_4', 'lagX_3','lagX_2', 'lagX_1',
#       'lagY_4','lagY_3', 'lagY_2', 'lagY_1',
#       'lagV_4', 'lagV_3', 'lagV_2', 'lagV_1'
#       ]]


new_data_co = new_data[['lagX_20', 'lagX_19', 'lagX_18', 'lagX_17', 'lagX_16',
      'lagX_15', 'lagX_14', 'lagX_13', 'lagX_12', 'lagX_11', 'lagX_10',
      'lagX_9', 'lagX_8', 'lagX_7', 'lagX_6', 'lagX_5', 'lagX_4', 'lagX_3',
      'lagX_2', 'lagX_1',
      'lagY_20', 'lagY_19', 'lagY_18', 'lagY_17',
      'lagY_16', 'lagY_15', 'lagY_14', 'lagY_13', 'lagY_12', 'lagY_11',
      'lagY_10', 'lagY_9', 'lagY_8', 'lagY_7', 'lagY_6', 'lagY_5', 'lagY_4',
      'lagY_3', 'lagY_2', 'lagY_1',
      'lagV_20', 'lagV_19', 'lagV_18',
      'lagV_17', 'lagV_16', 'lagV_15', 'lagV_14', 'lagV_13', 'lagV_12',
      'lagV_11', 'lagV_10', 'lagV_9', 'lagV_8', 'lagV_7', 'lagV_6', 'lagV_5',
      'lagV_4', 'lagV_3', 'lagV_2', 'lagV_1'
      ]]


In [ ]:
new_data[new_data['new_bounce']==1]

,index,x,y,V,bounce,hit,lagX_20,lagX_19,lagX_18,lagX_17,...,lagV_9,lagV_8,lagV_7,lagV_6,lagV_5,lagV_4,lagV_3,lagV_2,lagV_1,new_bounce
67,67,726.181818,448.636364,17.826878,0,0,687.111111,687.222222,687.333333,687.444444,...,17.146008,16.797242,17.447636,17.423483,17.682931,17.545369,17.678090,17.824236,17.665677,1
88,88,756.552632,479.868421,2.990229,0,0,729.363636,732.545455,735.727273,738.909091,...,3.120943,3.147065,3.079501,3.158201,3.130308,3.089944,3.137675,3.037740,3.004563,1
129,129,712.000000,398.000000,7.428548,0,0,753.789474,753.657895,753.526316,753.394737,...,20.915351,20.754222,19.625146,20.120089,20.777047,20.445942,20.737598,20.788856,19.870125,1
248,248,523.000000,110.000000,7.010440,0,0,632.000000,620.000000,614.076923,608.153846,...,19.265672,19.505100,19.955669,25.934564,21.342978,20.990236,21.289781,20.597456,16.375096,1
380,380,663.000000,117.000000,22.801273,0,0,623.000000,630.000000,633.000000,635.076923,...,9.583125,9.240693,9.392367,9.235424,14.661901,19.994823,19.495793,19.259315,19.187081,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13068,13068,711.571429,107.571429,6.697269,0,0,582.000000,600.000000,616.000000,632.000000,...,35.733448,36.055631,20.600619,23.235041,24.026392,22.673733,24.528188,13.998415,6.664108,1
13120,13120,208.000000,294.000000,41.888971,0,0,487.000000,473.000000,457.333333,441.666667,...,52.767344,49.827139,38.879115,39.169633,48.856550,39.423527,46.642286,44.787134,46.579018,1
13187,13187,570.285714,167.428571,34.061961,0,0,684.000000,674.000000,665.000000,656.000000,...,35.355189,37.090565,20.679019,23.936978,30.321061,33.130364,33.495908,34.096504,34.492774,1
13270,13270,576.500000,145.500000,6.160839,0,0,628.000000,623.000000,595.666667,568.333333,...,16.038581,19.029638,19.450443,19.413528,6.123145,6.142147,6.221030,6.265140,6.191996,1


In [ ]:
split_point = int(len(new_data_co)*0.9)

In [ ]:
new_train_X = new_data_co[:split_point]
new_test_X = new_data_co[split_point:]
new_train_Y = new_data['new_bounce'][:split_point]
new_test_Y = new_data['new_bounce'][split_point:]

In [ ]:
cb = CatBoostRegressor()

In [ ]:
cb.fit(new_train_X, new_train_Y)

Learning rate set to 0.060644
0:	learn: 0.1105315	total: 18.7ms	remaining: 18.7s
1:	learn: 0.1103524	total: 36.2ms	remaining: 18.1s
2:	learn: 0.1101615	total: 53.7ms	remaining: 17.9s
3:	learn: 0.1099028	total: 71.4ms	remaining: 17.8s
4:	learn: 0.1097343	total: 88.9ms	remaining: 17.7s
5:	learn: 0.1095796	total: 106ms	remaining: 17.6s
6:	learn: 0.1093999	total: 124ms	remaining: 17.6s
7:	learn: 0.1093285	total: 141ms	remaining: 17.5s
8:	learn: 0.1091127	total: 159ms	remaining: 17.5s
9:	learn: 0.1090315	total: 180ms	remaining: 17.8s
10:	learn: 0.1088775	total: 198ms	remaining: 17.8s
11:	learn: 0.1087242	total: 215ms	remaining: 17.7s
12:	learn: 0.1086601	total: 232ms	remaining: 17.6s
13:	learn: 0.1084519	total: 250ms	remaining: 17.6s
14:	learn: 0.1082506	total: 270ms	remaining: 17.7s
15:	learn: 0.1080913	total: 290ms	remaining: 17.8s
16:	learn: 0.1079503	total: 308ms	remaining: 17.8s
17:	learn: 0.1077714	total: 325ms	remaining: 17.7s
18:	learn: 0.1077169	total: 344ms	remaining: 17.8s
19:	le

In [ ]:
cb.predict(new_test_X)[0]

-0.00010525711229112701

In [ ]:
pred_thresh = 0.05
pred_bin = []

for i in range(len(new_test_X)):
  results = cb.predict(new_test_X)
  if results[i] >= pred_thresh:
    pred_bin.append(1)
  else:
    pred_bin.append(0)

confusion_matrix(pred_bin, new_test_Y)

array([[1224,   10],
       [  91,   11]])

In [ ]:
# cb.save_model('cbr_10_newdata')

In [ ]:
cb_final = CatBoostRegressor()

In [ ]:
cb_final.fit(new_data_co, new_data['new_bounce'])

Learning rate set to 0.061662
0:	learn: 0.1119162	total: 57.4ms	remaining: 57.3s
1:	learn: 0.1117165	total: 120ms	remaining: 59.8s
2:	learn: 0.1115091	total: 174ms	remaining: 57.8s
3:	learn: 0.1113412	total: 235ms	remaining: 58.5s
4:	learn: 0.1111281	total: 286ms	remaining: 57s
5:	learn: 0.1109357	total: 352ms	remaining: 58.4s
6:	learn: 0.1107710	total: 409ms	remaining: 58s
7:	learn: 0.1105677	total: 462ms	remaining: 57.3s
8:	learn: 0.1104021	total: 512ms	remaining: 56.4s
9:	learn: 0.1102435	total: 572ms	remaining: 56.7s
10:	learn: 0.1101143	total: 651ms	remaining: 58.5s
11:	learn: 0.1099858	total: 718ms	remaining: 59.1s
12:	learn: 0.1098590	total: 752ms	remaining: 57.1s
13:	learn: 0.1097169	total: 799ms	remaining: 56.3s
14:	learn: 0.1096381	total: 841ms	remaining: 55.2s
15:	learn: 0.1094770	total: 879ms	remaining: 54s
16:	learn: 0.1093392	total: 916ms	remaining: 53s
17:	learn: 0.1091504	total: 945ms	remaining: 51.5s
18:	learn: 0.1089573	total: 1.02s	remaining: 52.6s
19:	learn: 0.10882

In [ ]:
cb_final.save_model('cb_newdata_all')